
http is a package that collects several modules for working with the HyperText Transfer Protocol:

  - http.client is a low-level HTTP protocol client; for high-level URL opening use urllib.request
  - http.server contains basic HTTP server classes based on socketserver
  - http.cookies has utilities for implementing state management with cookies
  - http.cookiejar provides persistence of cookies

In [1]:
from http.server import HTTPServer

In [2]:
%%writefile hello.py
import http.server

class myresponse(http.server.SimpleHTTPRequestHandler):
   def do_GET(s):
       s.wfile.write("<body>Hello!</body>".encode("UTF-8"))

httpd = http.server.HTTPServer(("localhost", 8084), myresponse)
httpd.serve_forever()

Overwriting hello.py


In [7]:
%%writefile httpd.py
import time
import http.server


HOST_NAME = 'localhost' # !!!REMEMBER TO CHANGE THIS!!!
PORT_NUMBER = 8090 # Maybe set this to 9000.


def s2b(s):
    return s.encode("UTF-8")

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_HEAD(s):
        s.send_response(200)
        s.send_header(s2b("Content-type"), s2b("text/html"))
        s.end_headers()

    def do_GET(s):
        """Respond to a GET request."""
        s.send_response(200)
        s.send_header(s2b("Content-type"), s2b("text/html"))
        s.end_headers()
        s.wfile.write(s2b("<html><head><title>Title goes here.</title></head>"))
        s.wfile.write(s2b("<body><p>This is a <b>test.</b></p>"))
        # If someone went to "http://something.somewhere.net/foo/bar/",
        # then s.path equals "/foo/bar/".
        s.wfile.write(s2b("<p>You accessed path: %s</p>" % s.path))
        s.wfile.write(s2b("</body></html>"))

if __name__ == '__main__':
    server_class = http.server.HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime()), "Server Starts - %s:%s" % (HOST_NAME, PORT_NUMBER)
    httpd.serve_forever()
    #except KeyboardInterrupt:
    #    pass
    httpd.server_close()
    print(time.asctime()), "Server Stops - %s:%s" % (HOST_NAME, PORT_NUMBER)


Overwriting httpd.py


In [2]:
%%writefile cp1.py
import cherrypy

PORTNUM = 8093

class WelcomePage:
    def greetUser(self, name = None):
        if name:
            # Greet the user!
            return "Hey %s, what's up?" % name
        else:
            return 'call me like <i>http://localhost:{}/greetUser?name=Josh</i>'.format(PORTNUM)
    greetUser.exposed = True

cherrypy.config.update({"server.socket_port": PORTNUM})

cherrypy.quickstart(WelcomePage())

Overwriting cp1.py


In [2]:
%%writefile cp2.py
"""
Tutorial - Passing variables

This tutorial shows you how to pass GET/POST variables to methods.
"""
import os
import cherrypy


class WelcomePage:

    @cherrypy.expose
    def index(self):
        # Ask for the user's name.
        return '''
            <form action="greetUser" method="GET">
            What is your name?
            <input type="text" name="name" />
            <input type="submit" />
            </form>'''
    
    @cherrypy.expose
    def greetUser(self, name = None):
        # CherryPy passes all GET and POST variables as method parameters.
        # It doesn't make a difference where the variables come from, how
        # large their contents are, and so on.
        #
        # You can define default parameter values as usual. In this
        # example, the "name" parameter defaults to None so we can check
        # if a name was actually specified.
        
        if name:
            # Greet the user!
            return "Hey %s, what's up?" % name
        else:
            if name is None:
                # No name was specified
                return 'Please enter your name <a href="./">here</a>.'
            else:
                return 'No, really, enter your name <a href="./">here</a>.'
        
    def secret(self):
        return "here's my password: blah!"

if __name__ == '__main__':
    # CherryPy always starts with app.root when trying to map request URIs
    # to objects, so we need to mount a request handler root. A request
    # to '/' will be mapped to HelloWorld().index().
    cherrypy.config.update({"server.socket_port": 9010})

    cherrypy.quickstart(WelcomePage())
else:
    # This branch is for the test suite; you can ignore it.
    cherrypy.tree.mount(WelcomePage())


Overwriting cp2.py


## Restful apps

Flask-Restless provides simple generation of ReSTful APIs for database models defined using SQLAlchemy (or Flask-SQLAlchemy). The generated APIs send and receive messages in JSON format.

http://flask-restless.readthedocs.io/en/stable/

In [ ]:
!pip install flask-restless

In [3]:
%%writefile rest.py
import flask
import flask.ext.sqlalchemy
import flask.ext.restless

app = flask.Flask(__name__)
app.config["DEBUG"] = True
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:////tmp/test.db"

db = flask.ext.sqlalchemy.SQLAlchemy(app)

class Member(db.Model):
        # __tablename __ = "newsletter_members"
        id = db.Column(db.Integer,primary_key=True)
        last_name = db.Column(db.String(50))
        first_name = db.Column(db.String(120))
        email = db.Column(db.String(120),unique=True)

db.create_all()

manager = flask.ext.restless.APIManager(app,flask_sqlalchemy_db=db)

manager.create_api(Member,methods=["GET","POST"])

app.run()

Overwriting rest.py


Now run it in the terminal...

In [6]:
import json  # import simplejson as json, if on Python 2.5
import requests  # python-requests is installable from PyPI...
newmember = {'last_name': 'Lincoln', 'first_name': 'Abe', 'email': 'alincoln@whitehouse.gov'}
r = requests.post('http://127.0.0.1:5000/api/member', data=json.dumps(newmember),
                   headers={'content-type': 'application/json'})

In [8]:
r.status_code

201

In [9]:
r.headers['content-type']

'application/json, application/json'

In [10]:
r.text

'{\n  "email": "alincoln@whitehouse.gov", \n  "first_name": "Abe", \n  "id": 1, \n  "last_name": "Lincoln"\n}\n'

In [11]:
newid = json.loads(r.text)['id']

In [12]:
r = requests.get('http://127.0.0.1:5000/api/member/%s' % newid,
                  headers={'content-type': 'application/json'})

In [13]:
r.status_code, r.headers['content-type']

(200, 'application/json, application/json')

In [14]:
r.json()

{'email': 'alincoln@whitehouse.gov',
 'first_name': 'Abe',
 'id': 1,
 'last_name': 'Lincoln'}